In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('stopwords')




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [40]:
import os

# Function to read text from a file
def read_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Function to calculate similarity between report and each category
def calculate_similarity_with_categories(report_text, category_folder):
    category_similarity = {}

    # Get list of category files
    category_files = os.listdir(category_folder)

    for category_file in category_files:

        category_file_path = os.path.join(category_folder, category_file)
        print(category_file_path)
        if category_file_path == "/content/categories/.ipynb_checkpoints":
          print("helekjfdl")
          continue
        category_text = read_text_from_file(category_file_path)

        # Calculate similarity
        similarity_score = calculate_similarity(report_text, category_text)

        # Store similarity score for category
        category_name = os.path.splitext(category_file)[0]
        category_similarity[category_name] = similarity_score

    return category_similarity

# Function to display similarity results
def display_similarity_results(similarity_scores):
    total_similarity = sum(similarity_scores.values())
    for category, score in similarity_scores.items():
        percentage = (score / total_similarity) * 100 if total_similarity > 0 else 0
        print(f"{category}: {percentage:.2f}%")

# Example usage
report = "There is a problem in displaying user interface and user can not login , problem in authorization and validation is not working"
category_folder = "/content/categories"  # Folder containing category files

similarity_scores = calculate_similarity_with_categories(report, category_folder)
display_similarity_results(similarity_scores)


/content/categories/Frontend.txt
problem displaying user interface user login problem authorization validation working
frontend software application refers user interface users interact directly encompasses everything users see touch interact screens frontend development involves creating visual elements layout functionality application user interface frontend developers use variety technologies build responsive intuitive visually appealing interfaces html hypertext markup language provides structure webpage css cascading style sheets defines presentation layout javascript used add interactivity dynamic behavior frontend allowing users interact elements perform actions receive feedback real time addition core technologies frontend developers often utilize frameworks libraries streamline development enhance functionality popular frontend frameworks like react angular vue js provide reusable components state management routing capabilities allowing developers build complex user interface

In [13]:
import pandas as pd
df = pd.read_excel("train.xlsx")
df['class_name'].unique().tolist()

['Backend', 'Frontend', 'Security', 'Documentation', 'Performance']

In [35]:
import pandas as pd

# Function to read reports and labels from an Excel file
def read_reports_from_excel(file_path):
    df = pd.read_excel(file_path, nrows=1000)
    return df['report'].tolist(), df['class_name'].tolist()

# Function to calculate recall for each label
def calculate_recall(predicted_labels, true_labels, labels):
    recalls = {}
    for label in labels:
        true_positives = sum(1 for pred, true in zip(predicted_labels, true_labels) if pred == true == label)
        actual_positives = true_labels.count(label)
        recalls[label] = true_positives / actual_positives if actual_positives > 0 else 0
    return recalls

# Function to calculate similarity between report and each category
def calculate_similarity_with_categories(report_text, category_folder):
    category_similarity = {}

    # Get list of category files
    category_files = os.listdir(category_folder)

    for category_file in category_files:
        category_file_path = os.path.join(category_folder, category_file)
        if category_file_path == "categories/.ipynb_checkpoints":
          continue
        category_text = read_text_from_file(category_file_path)

        # Calculate similarity
        similarity_score = calculate_similarity(report_text, category_text)

        # Store similarity score for category
        category_name = os.path.splitext(category_file)[0]
        category_similarity[category_name] = similarity_score

    return category_similarity

# Function to determine the label with the highest similarity score
def determine_highest_similarity_label(similarity_scores):
    return max(similarity_scores, key=similarity_scores.get)

# Function to calculate accuracy
def calculate_accuracy(predicted_labels, true_labels):
    correct_predictions = sum(1 for pred, true in zip(predicted_labels, true_labels) if pred == true)
    total_reports = len(predicted_labels)
    accuracy = (correct_predictions / total_reports) * 100
    return accuracy

# Example usage
excel_file_path = "train.xlsx"  # Path to the Excel file
category_folder = "categories"  # Folder containing category files

# Read reports and labels from Excel file
reports, true_labels = read_reports_from_excel(excel_file_path)

# Get distinct labels
distinct_labels = list(set(true_labels))
predicted_labels = []
for report in reports:
    # Calculate similarity scores for the report
    similarity_scores = calculate_similarity_with_categories(report, category_folder)

    # Determine the predicted label with the highest similarity score
    predicted_label = determine_highest_similarity_label(similarity_scores)
    predicted_labels.append(predicted_label)

# Calculate accuracy
accuracy = calculate_accuracy(predicted_labels, true_labels)
print("Accuracy:", accuracy)

# Calculate recall
recall_scores = calculate_recall(predicted_labels, true_labels, distinct_labels)
for label, recall in recall_scores.items():
    print(f"Recall for {label}: {recall:.2f}")


Accuracy: 34.4
Recall for Backend: 0.15
Recall for Frontend: 0.59
Recall for Performance: 0.00
Recall for Documentation: 0.46
Recall for Security: 0.33


In [36]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    # Tokenize the text
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text.lower())



    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Stemming
    # stemmer = PorterStemmer()
    # stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
    print(' '.join(filtered_tokens))

    # Return preprocessed text as a single string
    return ' '.join(filtered_tokens)

def calculate_similarity(text1, text2):
    # Preprocess the texts
    preprocessed_text1 = preprocess_text(text1)
    preprocessed_text2 = preprocess_text(text2)

    # Calculate TF-IDF vectors
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([preprocessed_text1, preprocessed_text2])

    # Calculate cosine similarity
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

    return similarity

# Example usage
text1 = "The quick brown fox jumps over the lazy dog"
text2 = "A fast red fox leaps over the tired hound"
similarity_score = calculate_similarity(text1, text2)
print("Similarity score:", similarity_score)


quick brown fox jumps lazy dog
fast red fox leaps tired hound
Similarity score: 0.09193998174078082


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
